In [34]:
import os
import sys
import numpy as np
from collections import defaultdict, Counter
import time
from matplotlib import pyplot
import json5
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage
from scipy.spatial.distance import squareform

import importlib
import abxportinf
import abxportinf._grouper
import abxportinf._optimize
import abxportinf._vectorizer
import abxportinf.busdef
import abxportinf.main 
import abxportinf.util as util
from abxportinf.busdef import BusDef
from abxportinf._optimize import MatchCost

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [142]:
block_root = '/ip-block-designs'
#comp_json_path = os.path.join(
#    block_root,
#    'plda-pcie-controller',
#    'PLDA_ws.json5',
#)
comp_json_path = os.path.join(
    block_root,
    'ddr',
    'ddr.json5',
)
ports = abxportinf.get_ports_from_json5(comp_json_path)

Warning ('regHADDR', 'PAR_REG_ADDR_WIDTH + 2') not correctly parsed
Warning ('phy_reg_addr', '-(PAR_REG_ADDR_WIDTH)') not correctly parsed


In [153]:
bus_spec_rootdir = '/bus-defs/specs'
                                
print('loading bus defs from specs')
bus_defs = []                
for root, dirs, fnames in os.walk(bus_spec_rootdir):
    #print((root, dirs, files))  
    for fname in fnames:             
        spec_path = os.path.join(root, fname)
        if BusDef.is_spec_bus_def(spec_path):     
            print('  - loading ', spec_path)
            bus_defs.extend(
                BusDef.bus_defs_from_spec(spec_path)
            )                
print('  - done')                                            
print('loaded {} bus definitions from specs'.format(len(bus_defs)))
print('  - total req ports', sum([bd.num_req_ports for bd in bus_defs]))
print('  - total opt ports', sum([bd.num_opt_ports for bd in bus_defs]))

loading bus defs from specs
  - loading  /bus-defs/specs/amba.com/AMBA2/AHB/r3p0_1/AHB_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA5/AHB5Target/r0p0_0/AHB5Target_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA5/AHB5Initiator/r0p0_0/AHB5Initiator_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/AXI4Stream/r0p0_1/AXI4Stream_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/AXI4/r0p0_0/AXI4_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/AXI4/r0p0_0/AXI4_RO_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ACE-Lite/r0p0_0/ACE-Lite_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/APB4/r0p0_0/APB4_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ATB/r0p0_0/ATB_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ACP/r0p0_0/ACP_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ACE/r0p0_0/ACE_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA3/AHBLite/r2p0_0/AHBLite_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA3/APB/r2p0_0/APB_rtl.json5


In [154]:
from abxportinf._vectorizer import Vectorizer

pg, _, _ = abxportinf._grouper.get_port_grouper(ports)

#dname = 'axi0_WPARITY_EN'
dname = 'axi4_slv0_rready'

In [73]:
def tobits(mask):
    return ''.join(map(
        lambda b: '1' if b else '0',
        np.nditer(mask),
    ))

In [163]:
stime = time.time()

pg_bus_mappings = abxportinf.get_bus_matches(ports, list(bus_defs))

etime = time.time()
print('total time: {}s'.format(etime-stime))

pairing: 0, lcost:12.00(n:0.00;w:8;d:1), port group size: 22
       [('clk', 1, 1), ('controller_busy', 1, -1), ('controller_int', 1, -1), ('port_busy', 2, -1), ('regHADDR', None, 1)]
pairing: 1, lcost:15.00(n:0.00;w:15;d:0), port group size: 52
       [('axi0_ACLK', 1, 1), ('axi0_ARADDR', 37, 1), ('axi0_ARAPCMD', 1, 1), ('axi0_ARBURST', 2, 1), ('axi0_ARESETn', 1, 1)]
pairing: 2, lcost:15.00(n:0.00;w:15;d:0), port group size: 53
       [('axi0_ACLK', 1, 1), ('axi0_ARADDR', 37, 1), ('axi0_ARAPCMD', 1, 1), ('axi0_ARBURST', 2, 1), ('axi0_ARESETn', 1, 1)]
total time: 15.19500184059143s


In [169]:
dport = ('axi0_WPARITY_EN', 1, 1)
#dport = ('axi4_mst1_awready', 1, 1)
for port_group, bus_mappings in sorted(
    pg_bus_mappings,
    key=lambda x: x[1][0][0],
):
    #abxportinf.debug_bus_mapping(port_group, bus_mappings[1])
    print('port_group', list(sorted(port_group))[:5])
    for bm in bus_mappings:
        bd = bm[-1]
        print('  - ', bd.abstract_type, bd.driver_type)
        print('  - cost:{}, fcost:{}'.format(bm[0], bm[1]))
        if bd.abstract_type.name == 'AXI4_rtl':
            abxportinf.debug_bus_mapping(port_group, bm)
            
    
    #lbm = bus_mappings[0][-1]
    #lcost = bus_mappings[0][0]
    #if dport in port_group:
    #    print(len(port_group), lcost)
    #    print('  - ', lbm.abstract_type, lbm.driver_type)
    #    print(list(sorted(port_group)))
    #if lcost.dc < 2:
    #    print(port_group)
    #print(port_group)
    #if dport in port_group:
    #    #print('port group', len(port_group), lcost)
    #    #print(port_group)
    #    abxportinf.debug_bus_mapping(port_group, bus_mappings[0])
    #    print('size of port_group match', len(port_group))
    #    print('  - lcost', lcost)

port_group [('clk', 1, 1), ('controller_busy', 1, -1), ('controller_int', 1, -1), ('port_busy', 2, -1), ('regHADDR', None, 1)]
  -  {'vendor': 'amba.com', 'library': 'AMBA4', 'name': 'AXI4_RO_rtl', 'version': 'r0p0_0'} slave
  - cost:18.61(n:1.61;w:9;d:2), fcost:12.00(n:0.00;w:8;d:1)
  -  {'vendor': 'amba.com', 'library': 'AMBA5', 'name': 'AHB5Target_rtl', 'version': 'r0p0_0'} slave
  - cost:20.41(n:1.41;w:7;d:3), fcost:19.00(n:0.00;w:7;d:3)
  -  {'vendor': 'amba.com', 'library': 'AMBA5', 'name': 'AHB5Initiator_rtl', 'version': 'r0p0_0'} slave
  - cost:21.39(n:1.39;w:8;d:3), fcost:20.00(n:0.00;w:8;d:3)
  -  {'vendor': 'amba.com', 'library': 'AMBA3', 'name': 'AHBLite_rtl', 'version': 'r2p0_0'} slave
  - cost:26.21(n:1.21;w:9;d:4), fcost:25.00(n:0.00;w:9;d:4)
  -  {'vendor': 'amba.com', 'library': 'AMBA2', 'name': 'AHB_rtl', 'version': 'r3p0_1'} slave
  - cost:27.52(n:1.52;w:10;d:4), fcost:21.00(n:0.00;w:9;d:3)
port_group [('axi0_ACLK', 1, 1), ('axi0_ARADDR', 37, 1), ('axi0_ARAPCMD', 1, 

In [50]:
# for each bus mapping create a candidate component json5 entry
from itertools import chain

def get_abstract_type_str(bus_def):
    return '''{{vendor: '{}', library: '{}', name: '{}', version: '{}'}}'''.format(
        bus_def.abstract_type.vendor,
        bus_def.abstract_type.library,
        bus_def.abstract_type.name,
        bus_def.abstract_type.version,
    )
    
header =  '{\nbusInterfaces: [\n'
footer = ']\n}'
comp_obj_str = str(header)
    
seen_ports = set()
for i, (port_group, bus_mappings) in enumerate(sorted(
    pg_bus_mappings,
    key=lambda x: x[1][0][0],
)):
    #if i > 2:
    #    break
    # first bus mapping is the one with the lowest cost
    ccost, _, cmapping, _, cbus_def = bus_mappings[0]
    if ccost.dc > 1:
        continue
    bus_type_str = get_abstract_type_str(cbus_def)
    
    # name the bus according to the most frequent port word appearing (usually the prefix of all the signal names)
    mapping_name, _ = \
        Counter(chain(*map(lambda p: util.words_from_name(p[0]), port_group))).most_common(1)[0]
    
    # include all mapped bus_defs with the same direction mismatch cost as alternates
    alt_bus_defs = list(map(
        lambda bm: bm[-1],
        filter(
            lambda bm: bm[0].dc == ccost.dc,
            bus_mappings[1:],
        )
    ))
    alt_bus_def_str = ''
    if len(alt_bus_defs) > 0:
        alt_bus_def_str += 'alternateBusTypes: [\n'
        for bus_def in alt_bus_defs:
            alt_bus_def_str += get_abstract_type_str(bus_def) + ',\n'
        alt_bus_def_str += '\n],\n'
            
    # specify a conflict if the same physical port appears in multiple candidate bus mappings
    conflict_str = ''
    if set(port_group) & seen_ports:
        conflict_str += 'conflict: true\n,'
    seen_ports |= set(port_group)
    
    # format port map
    port_map_str = ''
    for pp, bp in cmapping.items():
        port_map_str += "{:s}: '{:s}',\n".format(pp[0], bp[0])
        #port_map_str += '[{:s},{:s}],\n'.format(str(list(pp)), str(list(bp)))
    
    comp_obj_str += \
'''{{
      name: '{}',
      interfaceMode: '{}',
      busType: {},
      {}
      {}
      abstractionTypes: [{{
        viewRef: 'RTLview',
        portMaps: {{
            {}
        }}
      }}], 
}},'''.format(
    mapping_name,
    cbus_def.driver_type,
    bus_type_str,
    alt_bus_def_str,
    conflict_str,
    port_map_str,
)
    
umap_ports = set(ports) - seen_ports
print('all ports', len(ports))
print('unmapped ports', len(umap_ports))
comp_obj_str += str(footer)
    
with open('test.json5', 'w') as fout:
    #fout.write(comp_obj_str)
    s = json5.dumps(json5.loads(comp_obj_str), indent=4)
    fout.write(s)
    

all ports 296
unmapped ports 144
